In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from preprocess import get_cifar100_datasets

train_ds, val_ds, test_ds = get_cifar100_datasets(image_size=(128, 128), batch_size=64)

In [14]:
def vgg19(input_shape=(128, 128, 3), num_classes=100):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(1e-4)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax')) 

    return model


In [15]:
model_VGG19 = vgg19()
model_VGG19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_VGG19.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_48 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_48 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_49 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_49 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 64, 64, 64)       

In [16]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model_VGG19.fit(
    train_ds,
    epochs=4,
    validation_data=val_ds,
    callbacks=[early_stop, lr_schedule],
)

Epoch 1/4
704/704 [==============================] - 221s 313ms/step - loss: 4.7744 - accuracy: 0.0093 - val_loss: 4.6597 - val_accuracy: 0.0100
Epoch 2/4
704/704 [==============================] - 215s 306ms/step - loss: 4.6453 - accuracy: 0.0087 - val_loss: 4.6321 - val_accuracy: 0.0100
Epoch 3/4
704/704 [==============================] - 215s 305ms/step - loss: 4.6255 - accuracy: 0.0083 - val_loss: 4.8466 - val_accuracy: 0.0100
Epoch 4/4
704/704 [==============================] - 214s 304ms/step - loss: 4.6184 - accuracy: 0.0081 - val_loss: 4.6120 - val_accuracy: 0.0100


In [17]:
test_loss, test_accuracy = model_VGG19.evaluate(test_ds, verbose=1)
print(f'Test accuracy: {test_accuracy:.4f}')

157/157 [==============================] - 14s 88ms/step - loss: 4.6120 - accuracy: 0.0100
Test accuracy: 0.0100
